In [1]:
from bs4 import BeautifulSoup as soup
from sqlite3 import connect
import requests

## Algorithms

In [2]:
def read_html(url = "https://books.toscrape.com/index.html"):
    """Read links in requests and returns a single html.parser"""
    # First step read the url as normal links
    libraries = soup(requests.get(url).text, "html.parser")
    
    # return html tags
    return libraries

In [3]:
def get_category_links(libraries):
    """Returns lis of full links of category`s book"""
    # Fixed domain (i.e part of link doesn`t change)
    DomainLinks = "https://books.toscrape.com/"
    FullLinks = [ ]
    # make list for all cateogry links 
    for link in libraries.find_all("li", attrs={"class":""}):
        FullLinks.append(DomainLinks + link.find("a").get("href"))
        
    # return that list
    return FullLinks

In [4]:
def make_single_url_chunked(single_url = "https://books.toscrape.com/index.html"):
    """Cut a single link"""
    # split the coming link
    chunked = single_url.split("/")[:-1]
    page_html_link = [page for page in chunked ] 
    # combine after chunking with forward slash '/'
    https = "/".join(page_html_link) + "/"
    
    # return parts of https link
    return https

In [5]:
def get_next_page(single_page):
    """expected html tags"""
    while single_page != '':
        if single_page.find("ul", attrs = {"class":"pager"}):
            # Try if single_page could used 'find' on it!
            try:
                return single_page.find("ul", attrs = {"class":"pager"}).find("li", attrs={"class":"next"}).find("a").get("href")
            # Catch errors and break the loop in display_scarping
            except Exception as e:
                break
        else:
            # returning "" if there`s no next page to scrap
            return ""

In [6]:
def url_merge(part1, part2):
    """Mergeing url the fixed part and nexts part"""
    if part2 != None:
        # return mergeed URL-> links webpage
        return part1 + part2
    else:
        # returning "" if there`s no next page to scrap
        return part1 + ""

## Writting on Database Tables

In [82]:
def write_on_tables(page_html):
    """write on books table the following details (names, rates, prices)
    the type of category comes from outside function write_on_cat_table()"""
    
    numerical_rates = {"One":1, "Two":2, "Three":3, "Four":4, "Five":5}
    category = page_html.find("title").get_text().split("|")[0].strip()
    
    for row in page_html.find_all("article" , attrs= {"class":"product_pod"}):
        names = row.find("h3").find("a").get("title")
        rates = numerical_rates[row.find("p").get("class")[1]]
        prices = float(row.find_all("p", attrs={"class":"price_color"})[0].get_text()[2:])

        connection.execute("INSERT INTO books (book_name, rates, price) values (?,?,?)", [names, rates, prices]) 
        connection.commit()
         
        book_id = connection.execute("SELECT book_id FROM books").fetchall()[-1][0]
        wite_on_cat_table(category, book_id)
        

In [83]:
def wite_on_cat_table(category, identity_book):
    """write type of category of specific book into categories table"""
    connection.execute("INSERT INTO categories (category, book_id) values (?, ?)", [category, identity_book])
    connection.commit()

## Main function "Displaying scraping"

In [89]:
def display_scraping(first_reading):
    """The main function starts scraping"""
    for single_link in get_category_links(first_reading):
        step2 = read_html(single_link)
        write_on_tables(step2)

        lp_step3 = get_next_page(step2)
        fp_step4 = make_single_url_chunked(single_link)

        step5 = url_merge(fp_step4, lp_step3)

        while get_next_page(step2):
            lp_step3 = get_next_page(step2)
            fp_step4 = make_single_url_chunked(single_link)
            step5 = url_merge(fp_step4, lp_step3)

            step2 = read_html(step5)
            write_on_tables(step2)
    print("The webpage has been scraped into database 'library' successfully ")

In [90]:
# connect Database -> 'library'
connection = connect("library.db")

# 2 functions for scraping and recording on DB
first_reading = read_html()
display_scraping(first_reading)

# close DB after recording on it
connection.close()

The webpage has been scraped into database 'library' successfully 
